In [1]:
!pip install numpy matplotlib pandas opencv-python

In [2]:
import os
from glob import glob # extract path of each file
import pandas as pd # data preprocessing
from xml.etree import ElementTree as et # parse information from XML
from functools import reduce

In [3]:
!unzip "/content/drive/MyDrive/1_datapreparation.zip" -d "/content/drive/My Drive/dataset"


Archive:  /content/drive/MyDrive/1_datapreparation.zip
   creating: /content/drive/My Drive/dataset/1_datapreparation/
   creating: /content/drive/My Drive/dataset/1_datapreparation/data_images/
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec01670c4ff2b.jpg  
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec01670c4ff2b.xml  
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e7eec37a.jpg  
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e7eec37a.xml  
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005db312f2.jpg  
  inflating: /content/drive/My Drive/dataset/1_datapreparation/data_images/00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005db312f2.x

In [12]:
# step-1: get path of each xml file
xmlfiles = glob('/content/drive/MyDrive/dataset/1_datapreparation/data_images/*.xml')
# replace \\ with /
replace_text = lambda x: x.replace('\\','/')
xmlfiles = list(map(replace_text,xmlfiles))

In [13]:
xmlfiles

['/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec01670c4ff2b.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e7eec37a.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005db312f2.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000099-Copy_jpg.rf.f2a0b66e15417a669de08d89d5f66a70.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000115-Copy_jpg.rf.7abf4be2c7fa5108680a46bbfc2eeee6.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000129-Copy_jpg.rf.213de334f1597f8aa68086603f49a844.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000135-Copy_jpg.rf.c9f96c67c23c7dbf837908f72fded406.xml',
 '/content/drive/MyDrive/dataset/1_datapreparation/data_images/00000141-Copy_jpg.rf.e6c329ebf6f639596d6f8add3529c2c2.xml',
 '/content/drive

In [14]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])

    return parser

In [15]:
parser_all = list(map(extract_text,xmlfiles))

In [16]:
data = reduce(lambda x, y : x+y,parser_all)

In [17]:
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [18]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,458,497,54,122
1,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,144,185,46,118
2,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,228,319,195,353
3,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,329,398,177,334
4,00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005...,640,640,face,124,219,63,240


In [19]:
df.shape

(5112, 8)

In [20]:
df['name'].value_counts()

,count
name,
face,5112


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112 entries, 0 to 5111
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  5112 non-null   object
 1   width     5112 non-null   object
 2   height    5112 non-null   object
 3   name      5112 non-null   object
 4   xmin      5112 non-null   object
 5   xmax      5112 non-null   object
 6   ymin      5112 non-null   object
 7   ymax      5112 non-null   object
dtypes: object(8)
memory usage: 319.6+ KB


In [22]:
# type conversion
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112 entries, 0 to 5111
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  5112 non-null   object
 1   width     5112 non-null   int64 
 2   height    5112 non-null   int64 
 3   name      5112 non-null   object
 4   xmin      5112 non-null   int64 
 5   xmax      5112 non-null   int64 
 6   ymin      5112 non-null   int64 
 7   ymax      5112 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 319.6+ KB


In [23]:
# center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
# w
df['w'] = (df['xmax']-df['xmin'])/df['width']
# h
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [24]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,458,497,54,122,0.746094,0.137500,0.060937,0.106250
1,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,144,185,46,118,0.257031,0.128125,0.064062,0.112500
2,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,228,319,195,353,0.427344,0.428125,0.142187,0.246875
3,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,329,398,177,334,0.567969,0.399219,0.107813,0.245312
4,00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005...,640,640,face,124,219,63,240,0.267969,0.236719,0.148438,0.276562


In [25]:
images = df['filename'].unique()

In [26]:
len(images)

1635

In [27]:
# 80% train and 20% test
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [28]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [29]:
len(img_train), len(img_test)

(1308, 327)

In [30]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [31]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,458,497,54,122,0.746094,0.137500,0.060937,0.106250
1,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,144,185,46,118,0.257031,0.128125,0.064062,0.112500
2,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,228,319,195,353,0.427344,0.428125,0.142187,0.246875
3,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,329,398,177,334,0.567969,0.399219,0.107813,0.245312
4,00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005...,640,640,face,124,219,63,240,0.267969,0.236719,0.148438,0.276562


In [32]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
9,00000115-Copy_jpg.rf.7abf4be2c7fa5108680a46bbf...,640,640,face,202,262,145,269,0.362500,0.323437,0.093750,0.193750
10,00000115-Copy_jpg.rf.7abf4be2c7fa5108680a46bbf...,640,640,face,436,454,204,236,0.695312,0.343750,0.028125,0.050000
11,00000129-Copy_jpg.rf.213de334f1597f8aa68086603...,640,640,face,299,340,141,223,0.499219,0.284375,0.064062,0.128125
12,00000129-Copy_jpg.rf.213de334f1597f8aa68086603...,640,640,face,354,396,99,176,0.585938,0.214844,0.065625,0.120313
13,00000129-Copy_jpg.rf.213de334f1597f8aa68086603...,640,640,face,198,239,200,272,0.341406,0.368750,0.064062,0.112500


In [35]:
# label encoding
def label_encoding(x):
    labels = {'face':0}
    return labels[x]

In [36]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

<ipython-input-36-429437bd50d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
<ipython-input-36-429437bd50d8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [37]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,458,497,54,122,0.746094,0.137500,0.060937,0.106250,0
1,00000024-Copy_jpg.rf.015e1cc8e7a4be02254ec0167...,640,640,face,144,185,46,118,0.257031,0.128125,0.064062,0.112500,0
2,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,228,319,195,353,0.427344,0.428125,0.142187,0.246875,0
3,00000058-Copy_jpg.rf.a0e7963c2843d436c5886d08e...,640,640,face,329,398,177,334,0.567969,0.399219,0.107813,0.245312,0
4,00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005...,640,640,face,124,219,63,240,0.267969,0.236719,0.148438,0.276562,0
5,00000087-Copy_jpg.rf.da6df6aaf8b403a6dcb036005...,640,640,face,435,529,74,251,0.753125,0.253906,0.146875,0.276562,0
6,00000099-Copy_jpg.rf.f2a0b66e15417a669de08d89d...,640,640,face,233,333,208,300,0.442188,0.396875,0.156250,0.143750,0
7,00000099-Copy_jpg.rf.f2a0b66e15417a669de08d89d...,640,640,face,401,492,251,339,0.697656,0.460938,0.142187,0.137500,0
8,00000099-Copy_jpg.rf.f2a0b66e15417a669de08d89d...,640,640,face,215,300,82,161,0.402344,0.189844,0.132812,0.123438,0
20,00000141-Copy_jpg.rf.e6c329ebf6f639596d6f8add3...,640,640,face,93,278,153,316,0.289844,0.366406,0.289062,0.254688,0


In [38]:
import os
from shutil import move

In [39]:


root_folder = "/content/drive/MyDrive/dataset/1_datapreparation/data_images"

train_folder = os.path.join(root_folder, "train")
test_folder = os.path.join(root_folder, "test")

# Create the directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

print("Folders created successfully!")


Folders created successfully!


In [40]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [47]:
#groupby_obj_train.get_group('000009.jpg').set_index('filename').to_csv('sample.txt',index=False,header=False)
# save each image in train/test folder and repective labels in .txt

import shutil

def save_data(filename, folder_path, group_obj):
    # Define the correct source image path
    src = os.path.join('/content/drive/MyDrive/dataset/1_datapreparation/data_images', filename)
    dst = os.path.join(folder_path, filename)

    # Move the image file to the destination folder (only if it exists)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"Warning: {src} not found!")

    # Save the label file
    text_filename = os.path.join(folder_path, os.path.splitext(filename)[0] + '.txt')

    # Check if the filename exists in group_obj before accessing
    if filename in group_obj.groups:
        group_obj.get_group(filename).set_index('filename').to_csv(
            text_filename, sep=' ', index=False, header=False
        )
    else:
        print(f"Warning: No label data found for {filename}")


In [48]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [49]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

,0
0,None
1,None
2,None
3,None
4,None
...,...
1303,None
1304,None
1305,None
1306,None


In [50]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

,0
0,None
1,None
2,None
3,None
4,None
...,...
322,None
323,None
324,None
325,None
